In [1]:
#import all packages necessary for the program

import numpy as np          #classics
import geopandas as gpd

import ee                   #earth engine
import geemap
import geeet
from geeet.eepredefined import landsat

import arcpy               #arcgis
from arcpy import env

import time                 #time management

In [2]:
#initialize earth engine
#ee.Authenticate() #uncomment this line if running for the first time
ee.Initialize()


In [4]:
#Define area of interest
region = ee.FeatureCollection("projects/ee-redwall6152/assets/DC_bound")

#Define time range
start_date = '2022-03-01'
end_date = '2022-10-31'
year = '2022'



# Define a custom workflow (TSEB model + LE extrapolation)
workflow = [
    geeet.tseb.tseb_series,
    landsat.extrapolate_LE    # this adds the "ET" band, in mm/day
]

# Create Landsat TSEB collection with ERA5 data 

landsat_era5_tseb_collection = landsat.mapped_collection(
    workflow,
    date_start = start_date,
    date_end = end_date,
    region = region,
    era5 = True,
    timeZone = 'America/Boise',
)

print('There are', landsat_era5_tseb_collection.getInfo()['features'].__len__(), 'images in the collection.')

There are 109 images in the collection.


In [ ]:
#Check dates of collected images
dates = landsat_era5_tseb_collection.aggregate_array('system:time_start').getInfo()
readable_dates = [time.strftime('%Y-%m-%d', time.gmtime(d/1000)) for d in dates]
print('Dates of images in collection:', readable_dates)

#Check file names
file_names = landsat_era5_tseb_collection.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()
print('File names in collection:', file_names)



Dates of images in collection: ['2013-06-22', '2013-07-08', '2013-08-09', '2013-08-25', '2013-09-10', '2013-04-11', '2013-04-27', '2013-05-13', '2013-06-14', '2013-06-30', '2013-08-01', '2013-10-04', '2013-10-20']
File names in collection: ['LE07_L2SP_041030_20130622_20200907_02_T1', 'LE07_L2SP_041030_20130708_20200907_02_T1', 'LE07_L2SP_041030_20130809_20200907_02_T1', 'LE07_L2SP_041030_20130825_20200907_02_T1', 'LE07_L2SP_041030_20130910_20200907_02_T1', 'LC08_L2SP_041030_20130411_20200912_02_T1', 'LC08_L2SP_041030_20130427_20200913_02_T1', 'LC08_L2SP_041030_20130513_20200912_02_T1', 'LC08_L2SP_041030_20130614_20200912_02_T1', 'LC08_L2SP_041030_20130630_20200912_02_T1', 'LC08_L2SP_041030_20130801_20200912_02_T1', 'LC08_L2SP_041030_20131004_20200913_02_T1', 'LC08_L2SP_041030_20131020_20200912_02_T1']


In [9]:
#Define a clipping function
def clip_to_region(img):
    return img.clip(region)

#Clip the collection to the region
landsat_era5_tseb_clipped = landsat_era5_tseb_collection.map(clip_to_region)

print('There are', landsat_era5_tseb_clipped.getInfo()['features'].__len__(), 'images in the clipped collection.')

# Function to compute cloud fraction over ROI
def add_cloud_cover_roi(img):
    qa = img.select('QA_PIXEL')
    # 0 = clear, 1 = cloud/shadow
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    total = cloud_mask.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=region.geometry(),
        scale=30,
        maxPixels=1e9
    ).getNumber('QA_PIXEL')
    clear = cloud_mask.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region.geometry(),
        scale=30,
        maxPixels=1e9
    ).getNumber('QA_PIXEL')
    cloud_fraction = ee.Number(1).subtract(clear.divide(total)).multiply(100)
    return img.set('CLOUD_COVER_ROI', cloud_fraction)

# Apply cloud fraction calculation to your clipped collection
landsat_with_clouds = landsat_era5_tseb_clipped.map(add_cloud_cover_roi)


# Filter for Landsat 8 or 9
landsat_8_9 = landsat_with_clouds.filter(
    ee.Filter.inList('SPACECRAFT_ID', ['LANDSAT_8', 'LANDSAT_9'])
)

print('There are', landsat_8_9.size().getInfo(), 'images from Landsat 8 or 9.')

# Filter for cloud cover below threshold (e.g., 80%)
cloud_threshold = 10
filtered_collection = landsat_8_9.filter(
    ee.Filter.lt('CLOUD_COVER_ROI', cloud_threshold)
)

print('There are', filtered_collection.size().getInfo(), 'Landsat 8/9 images with cloud cover below', cloud_threshold, '%.')

There are 109 images in the clipped collection.
There are 85 images from Landsat 8 or 9.
There are 38 Landsat 8/9 images with cloud cover below 10 %.


In [7]:
#Get info on the new band names in the collection
landsat_era5_tseb_clipped.first().bandNames().getInfo()


['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B7',
 'SR_ATMOS_OPACITY',
 'SR_CLOUD_QA',
 'ST_B6',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT',
 'NDVI',
 'albedo',
 'radiometric_temperature',
 'cloud_cover',
 'surface_pressure',
 'air_temperature',
 'dewpoint_temperature',
 'u_component_of_wind_10m',
 'v_component_of_wind_10m',
 'surface_solar_radiation_downwards_hourly',
 'surface_thermal_radiation_downwards_hourly',
 'wind_speed',
 'solar_radiation',
 'thermal_radiation',
 'Tc',
 'Ts',
 'Tac',
 'Hc',
 'Hs',
 'LEc',
 'LEs',
 'Ra',
 'Rs',
 'Rx',
 'Ustar',
 'alphaPT',
 'iteration',
 'LE',
 'H',
 'G',
 'Rn',
 'Rns',
 'Rnc',
 'ET']

In [119]:
#Create a seasonal composite of the ET band
landsat_clipped_et = landsat_era5_tseb_clipped.select('ET')
landsat_et_composite = landsat_clipped_et.mean()


In [120]:
# === IMPORT PACKAGES ===
import time
import os
import io
import zipfile
import arcpy

from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload

# === USER CONFIGURATION ===
SERVICE_ACCOUNT_FILE = r"C:\School\thesis_stuff\DryCreekLandsat\gee-drive-access-beed7fc2e972.json"
LOCAL_EXPORT_FOLDER = r"C:\School\Thesis_stuff\DryCreekLandsat\gee_exports"
GDRIVE_FOLDER = 'gee_exports'
EXPORT_PREFIX = 'DryCreek_ET' + year
EXPORT_TYPE = 'image'   # 'image' or 'table'

# === INITIALIZE EARTH ENGINE ===
ee.Initialize()

# === CREATE EXPORT TASK ===
if EXPORT_TYPE == 'image':
    task = ee.batch.Export.image.toDrive(
        image=landsat_et_composite,
        description=EXPORT_PREFIX,
        folder=GDRIVE_FOLDER,
        fileNamePrefix=EXPORT_PREFIX,
        region=region.geometry().bounds(),
        scale=30,
        crs='EPSG:32611'
    )
elif EXPORT_TYPE == 'table':
    fc = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
    task = ee.batch.Export.table.toDrive(
        collection=fc,
        description=EXPORT_PREFIX,
        folder=GDRIVE_FOLDER,
        fileNamePrefix=EXPORT_PREFIX,
        fileFormat='SHP'
    )
else:
    raise ValueError("EXPORT_TYPE must be 'image' or 'table'.")

# === START AND MONITOR TASK ===
task.start()
print("🚀 GEE export started... waiting for completion.")

while task.status()['state'] in ['READY', 'RUNNING']:
    print("⏳ Task state:", task.status()['state'])
    time.sleep(30)  # check every 30 seconds

status = task.status()
print("✅ Export complete:", status)

if status['state'] != 'COMPLETED':
    raise RuntimeError(f"Export failed: {status}")



    


🚀 GEE export started... waiting for completion.
⏳ Task state: READY
⏳ Task state: RUNNING
⏳ Task state: RUNNING
⏳ Task state: RUNNING
⏳ Task state: RUNNING
⏳ Task state: RUNNING
✅ Export complete: {'state': 'COMPLETED', 'description': 'DryCreek_ET2022', 'priority': 100, 'creation_timestamp_ms': 1761273098612, 'update_timestamp_ms': 1761273279656, 'start_timestamp_ms': 1761273109408, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1rj597hrdb1FQ3lPdUp_A__dFgUlMrl31'], 'attempt': 1, 'batch_eecu_usage_seconds': 448.31695556640625, 'id': 'DNBMY7H4DSPVPF7XQRF7XIUZ', 'name': 'projects/153414845949/operations/DNBMY7H4DSPVPF7XQRF7XIUZ'}


In [121]:
# === AUTHENTICATE GOOGLE DRIVE ===
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
drive_service = build('drive', 'v3', credentials=creds)

# === FIND FILE IN GOOGLE DRIVE ===
query = f"name contains '{EXPORT_PREFIX}'"
results = drive_service.files().list(q=query, fields="files(id, name)").execute()
files = results.get('files', [])

if not files:
    raise FileNotFoundError("❌ No exported file found in Google Drive.")

for f in files:
    file_id = f['id']
    file_name = f['name']
    print(f"⬇️ Downloading {file_name} from Google Drive...")

    local_path = os.path.join(LOCAL_EXPORT_FOLDER, file_name)
    os.makedirs(LOCAL_EXPORT_FOLDER, exist_ok=True)

    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, 'wb')
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f"  Progress: {int(status.progress() * 100)}%")

    fh.close()
    print(f"✅ Downloaded: {local_path}")

    # === UNZIP IF NEEDED ===
    if file_name.endswith(".zip"):
        with zipfile.ZipFile(local_path, 'r') as zip_ref:
            zip_ref.extractall(LOCAL_EXPORT_FOLDER)
        print(f"📂 Extracted contents to {LOCAL_EXPORT_FOLDER}")



⬇️ Downloading DryCreek_ET2022.tif from Google Drive...
  Progress: 100%
✅ Downloaded: C:\School\Thesis_stuff\DryCreekLandsat\gee_exports\DryCreek_ET2022.tif


In [ ]:
# === ADD DOWNLOADED FILE(S) TO ARCGIS MAP ===
print("🗺️ Adding exported layers to ArcGIS Pro project...")

aprx = arcpy.mp.ArcGISProject(r"C:\School\thesis_stuff\DryCreekLandsat\DryCreekLandsat\DryCreekLandsat.aprx")
m = aprx.activeMap

for root, dirs, files in os.walk(LOCAL_EXPORT_FOLDER):
    for file in files:
        if file.endswith((".tif", ".shp")):
            full_path = os.path.join(root, file)
            try:
                m.addDataFromPath(full_path)
                print(f"✅ Added to map: {file}")
            except Exception as e:
                print(f"⚠️ Could not add {file}: {e}")

print("🎉 All done! Layers from GEE are now added to your ArcGIS Pro map.")
]

for layer in layers_to_add:
    path = os.path.join(layer_folder, layer)
    if os.path.exists(path):
        m.addDataFromPath(path)
        print(f"✅ Added {layer}")
    else:
        print(f"⚠️ Missing layer: {layer}")

print("All requested layers processed.")

